In [1]:
import pandas as pd
import hillmaker as hm
from pandas import DataFrame
from pandas import Timestamp
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import Minute

In [2]:
import itertools

In [3]:
from hillmaker import bydatetime

In [4]:
help(bydatetime.make_bydatetime)

Help on function make_bydatetime in module hillmaker.bydatetime:

make_bydatetime(stops_df, infield, outfield, start_analysis, end_analysis, catfield, total_str='Total', bin_size_minutes=60, cat_to_exclude=None, totals=True, verbose=0)
    Create bydatetime table based on user inputs.
    
    This is the table from which summary statistics can be computed.
    
    Parameters
    ----------
    stops_df: DataFrame
        Stop data
    
    infield: string
        Name of column in stops_df to use as arrival datetime
    
    outfield: string
        Name of column in stops_df to use as departure datetime
    
    start_analysis: datetime
        Start date for the analysis
    
    end_analysis: datetime
        End date for the analysis
    
    catfield : string or List of strings, optional
        Column name(s) corresponding to the categories. If none is specified, then only overall occupancy is analyzed.
    
    total_str: string, default 'Total'
        Value to use for the to

In [4]:
file_stopdata = '../data/ShortStay2.csv'

scenario = 'iss17_test1'
in_fld_name = 'InRoomTS'
out_fld_name = 'OutRoomTS'
cat_fld_name = 'PatType'
start_analysis_dt = '1/1/1996'
end_analysis_dt = '3/30/1996 23:45'

# Optional inputs
tot_fld_name = 'SSU'
bin_size_minutes = 60

In [5]:
stops_df = pd.read_csv(file_stopdata, parse_dates=[in_fld_name, out_fld_name])

In [6]:
stops_df.tail(4)

,PatID,InRoomTS,OutRoomTS,PatType,Severity
59873,59874,1996-09-30 20:23:00,1996-09-30 21:30:00,IVT,2
59874,59875,1996-09-30 21:00:00,1996-09-30 22:45:00,CAT,1
59875,59876,1996-09-30 21:57:00,1996-09-30 22:40:00,IVT,2
59876,59877,1996-09-30 22:45:00,1996-09-30 23:35:00,CAT,1


In [21]:
rng_bydt = pd.date_range(start_analysis_dt, end_analysis_dt, freq=Minute(bin_size_minutes))

In [22]:
len(rng_bydt)

2160

In [26]:
pd.Series(rng_bydt).head()

0   1996-01-01 00:00:00
1   1996-01-01 01:00:00
2   1996-01-01 02:00:00
3   1996-01-01 03:00:00
4   1996-01-01 04:00:00
dtype: datetime64[ns]

In [9]:
catfield = ['PatType','Severity']

In [10]:
catfield[1]

'Severity'

In [11]:
categories = []
cat_to_exclude = [[],[]]
if cat_to_exclude is not None:
        for i in range(len(catfield)):
            categories.append(tuple([c for c in stops_df[catfield[i]].unique() if c not in cat_to_exclude[i]]))
else:
        for i in range(len(catfield)):
            categories.append(tuple([c for c in stops_df[catfield[i]].unique()]))


In [13]:
categories

[('IVT', 'MYE', 'CAT', 'ART', 'OTH'), (1, 2)]

In [28]:
numcombos = 1
for c in categories:
    numcombos = numcombos * len(c)
print(numcombos)

10


In [14]:
# Only include stops records that have their categories included
for i in range(len(catfield)):
    stops_df = stops_df[stops_df[catfield[i]].isin(categories[i])]

In [18]:
# Create a list of column names for the by datetime table and then an empty data frame based on these columns.
# The column names for the category fields are now the actual field names
columns = []
measures = ['datetime', 'arrivals', 'departures', 'occupancy']
for field in catfield:
    columns.append(field)

columns.extend(measures)

bydt_df = DataFrame(columns=columns)


In [19]:
print(columns)

['PatType', 'Severity', 'datetime', 'arrivals', 'departures', 'occupancy']


In [20]:
bydt_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 6 columns):
PatType       0 non-null object
Severity      0 non-null object
datetime      0 non-null object
arrivals      0 non-null object
departures    0 non-null object
occupancy     0 non-null object
dtypes: object(6)
memory usage: 0.0+ bytes


In [51]:
itertools.product(('IVT', 'MYE', 'CAT', 'ART', 'OTH'),(1,2))

In [56]:
for p in itertools.product(['IVT', 'MYE', 'CAT', 'ART', 'OTH'],[1,2]):
    print(p)

('IVT', 1)
('IVT', 2)
('MYE', 1)
('MYE', 2)
('CAT', 1)
('CAT', 2)
('ART', 1)
('ART', 2)
('OTH', 1)
('OTH', 2)


In [21]:
for p in itertools.product(*categories):
    print(p)
    print(*p)

('IVT', 1)
IVT 1
('IVT', 2)
IVT 2
('MYE', 1)
MYE 1
('MYE', 2)
MYE 2
('CAT', 1)
CAT 1
('CAT', 2)
CAT 2
('ART', 1)
ART 1
('ART', 2)
ART 2
('OTH', 1)
OTH 1
('OTH', 2)
OTH 2


In [23]:
print(*categories)

('IVT', 'MYE', 'CAT', 'ART', 'OTH') (1, 2)


In [36]:
for p in itertools.product(('IVT', 'MYE', 'CAT', 'ART', 'OTH'),(1,2)):
    print([*p],p[0],p[1])

['IVT', 1] IVT 1
['IVT', 2] IVT 2
['MYE', 1] MYE 1
['MYE', 2] MYE 2
['CAT', 1] CAT 1
['CAT', 2] CAT 2
['ART', 1] ART 1
['ART', 2] ART 2
['OTH', 1] OTH 1
['OTH', 2] OTH 2


In [27]:
len(itertools.product(*categories))

TypeError: object of type 'itertools.product' has no len()

In [19]:
df1 = pd.DataFrame({'B': ['B0', 'B1', 'B2', 'B3'],
   ...:                     'D': ['D0', 'D1', 'D2', 'D3']},
   ...:                     index=[0, 1, 2, 3])

In [19]:
df1 = pd.DataFrame({'B': ['B0', 'B1', 'B2', 'B3'],
   ...:                     'D': ['D0', 'D1', 'D2', 'D3']},
   ...:                     index=[0, 1, 2, 3])

In [20]:
df1

,B,D
0,B0,D0
1,B1,D1
2,B2,D2
3,B3,D3


In [21]:
df2 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
   ...:                     'C': ['C0', 'C1', 'C2', 'C3']},               
   ...:                     index=[0, 1, 2, 3])

In [22]:
df2

,A,C
0,A0,C0
1,A1,C1
2,A2,C2
3,A3,C3


In [23]:
pd.concat([df1,df2],axis=1)

,B,D,A,C
0,B0,D0,A0,C0
1,B1,D1,A1,C1
2,B2,D2,A2,C2
3,B3,D3,A3,C3


In [32]:
print(catfield)

['PatType', 'Severity']


In [58]:
rng_bydt = pd.Series(pd.date_range(start_analysis_dt, end_analysis_dt, freq=Minute(bin_size_minutes)))

In [39]:
itertools.product(*categories)

In [30]:
i=2
print(i)
i+=1
print(i)

2
3


In [59]:
len_bydt = len(rng_bydt)
all_cat_df = []
for p in itertools.product(*categories):
        i=0
        cat_df = DataFrame()
        j=0
        for c in [*p]:
            print(c)
            bydt_catdata = {catfield[j]: [c] * len_bydt}
            cat_df_cat = DataFrame(bydt_catdata, columns=[catfield[j]])
            j+=1
            #cat_df_cat.info()
            cat_df = pd.concat([cat_df, cat_df_cat],axis=1)
        all_cat_df.append(cat_df)
        i+=1

IVT
1
IVT
2
MYE
1
MYE
2
CAT
1
CAT
2
ART
1
ART
2
OTH
1
OTH
2


In [60]:
all_cat_df[1]

,PatType,Severity
0,IVT,2
1,IVT,2
2,IVT,2
3,IVT,2
4,IVT,2
5,IVT,2
6,IVT,2
7,IVT,2
8,IVT,2
9,IVT,2


In [35]:
cat_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2160 entries, 0 to 2159
Data columns (total 4 columns):
PatType                      0 non-null object
Severity                     0 non-null object
(IVT, MYE, CAT, ART, OTH)    2160 non-null object
(1, 2)                       2160 non-null int64
dtypes: int64(1), object(3)
memory usage: 84.4+ KB


In [61]:
bydt_data = {'datetime': rng_bydt, 'arrivals': [0.0] * len_bydt,
                     'departures': [0.0] * len_bydt, 'occupancy': [0.0] * len_bydt}

In [69]:
bydt_data['datetime'][10:20]

10   1996-01-01 10:00:00
11   1996-01-01 11:00:00
12   1996-01-01 12:00:00
13   1996-01-01 13:00:00
14   1996-01-01 14:00:00
15   1996-01-01 15:00:00
16   1996-01-01 16:00:00
17   1996-01-01 17:00:00
18   1996-01-01 18:00:00
19   1996-01-01 19:00:00
dtype: datetime64[ns]

In [70]:
bydt_data_df = DataFrame(bydt_data, columns=['datetime', 'arrivals', 'departures', 'occupancy'])

In [71]:
bydt_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2160 entries, 0 to 2159
Data columns (total 4 columns):
datetime      2160 non-null datetime64[ns]
arrivals      2160 non-null float64
departures    2160 non-null float64
occupancy     2160 non-null float64
dtypes: datetime64[ns](1), float64(3)
memory usage: 84.4 KB


In [72]:
bydt_data_df.head()

,datetime,arrivals,departures,occupancy
0,1996-01-01 00:00:00,0,0,0
1,1996-01-01 01:00:00,0,0,0
2,1996-01-01 02:00:00,0,0,0
3,1996-01-01 03:00:00,0,0,0
4,1996-01-01 04:00:00,0,0,0


In [73]:
bydt_df = DataFrame()
for cat_df in all_cat_df:
   
        #bydt_df_cat = DataFrame(bydt_data, columns=['category', 'datetime', 'arrivals', 'departures', 'occupancy'])
        bydt_df_cat = pd.concat([cat_df, bydt_data_df],axis=1)
        bydt_df = pd.concat([bydt_df, bydt_df_cat])

In [49]:
len_bydt

2160

In [64]:
bydt_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43200 entries, 0 to 2159
Data columns (total 6 columns):
PatType       21600 non-null object
Severity      21600 non-null float64
arrivals      21600 non-null float64
datetime      21600 non-null datetime64[ns]
departures    21600 non-null float64
occupancy     21600 non-null float64
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 2.3+ MB


In [74]:
bydt_df.head()

,PatType,Severity,datetime,arrivals,departures,occupancy
0,IVT,1,1996-01-01 00:00:00,0,0,0
1,IVT,1,1996-01-01 01:00:00,0,0,0
2,IVT,1,1996-01-01 02:00:00,0,0,0
3,IVT,1,1996-01-01 03:00:00,0,0,0
4,IVT,1,1996-01-01 04:00:00,0,0,0


In [76]:
bydt_df = DataFrame()
bydt_data = {'datetime': rng_bydt, 'arrivals': [0.0] * len_bydt,
                         'departures': [0.0] * len_bydt, 'occupancy': [0.0] * len_bydt}
bydt_data_df = DataFrame(bydt_data, columns=['datetime', 'arrivals', 'departures', 'occupancy'])

for cat_df in all_cat_df:
    bydt_df_cat = pd.concat([cat_df, bydt_data_df],axis=1)
    bydt_df = pd.concat([bydt_df, bydt_df_cat])

In [77]:
bydt_df.head()

,PatType,Severity,datetime,arrivals,departures,occupancy
0,IVT,1,1996-01-01 00:00:00,0,0,0
1,IVT,1,1996-01-01 01:00:00,0,0,0
2,IVT,1,1996-01-01 02:00:00,0,0,0
3,IVT,1,1996-01-01 03:00:00,0,0,0
4,IVT,1,1996-01-01 04:00:00,0,0,0


In [78]:
bydt_df.tail()

,PatType,Severity,datetime,arrivals,departures,occupancy
2155,OTH,2,1996-03-30 19:00:00,0,0,0
2156,OTH,2,1996-03-30 20:00:00,0,0,0
2157,OTH,2,1996-03-30 21:00:00,0,0,0
2158,OTH,2,1996-03-30 22:00:00,0,0,0
2159,OTH,2,1996-03-30 23:00:00,0,0,0
